In [1]:
# Sampling
# 1. Sample using HGT Sampler as outlined in the paper, using pyg implementations
# 2. The sampling is adapted to link prediction, by first sampling random supervision edges of which the nodes create the supervision nodes
# a. Dataset is divided across multiple dimensions:
#   a.1. Split into Train, Test split
#   a.2. Training only: Edges are split into those which are used solely for message passing and those solely used for supervision (70, 30). 
#        Because an expressive model (HGT) is used, this prevents the model from memorizing supervision edges by their appearance as message passing edges
#   a.4. Test edges use the Training Message passing Edges as well.
# b. For mini-batch sampling in the training phase, first x random edges are sampled as supervision edges. 
#    For the nodes of these supervision edges, we apply batch-wise HGT Sampling. Due to implementation limitations, for each supervision entity type, the hgt sampling is separate. 
# during sampling, also the reverse edge of the supervision edge is removed to avoid data leakage


# HGT Sampler (See Paper for further reference)
# The probablity of a neighbor node s to be sampled depends on the normalized degree of all its edge types connecting it to all source nodes
# If neighbor node s is connected to a and b by edge type r, and a has 2 neighbors through edge type r and b has 1 neighbor (node s) through edge type r, 
# then the sampling probablity of s is (1/2+1)**2 / 2**2, if it were connected through other edge types to the nodes as well, those degrees would be added to the numerator and denominator.
# Nodes are sampled without replacement.
# This sampling strategy creates more dense mini-batches, because neighbor nodes which are connected to multiple source nodes and by multiple relationship types are sampled more frequently.
# Therefore, training is sped up since less node representations have to be computed. Furthermore, as stated in the paper, the sampling method allows to sample a 
# similar number of neighbors for each edge type, because high-count edge types and low-count edge types are weighted equally. For each neighbor node type T, a fixed number n of nodes is sampled.





import os
import torch
from torch_geometric.data import HeteroData
from tqdm.auto import tqdm
import numpy as np

def add_reverse_edge_original_attributes_and_label_inplace(original_edge, reverse_edge):
    # add edge label and index and edge attr to reverse edge 
    if 'edge_attr' in original_edge:
        reverse_edge['edge_attr'] = original_edge['edge_attr']
    reverse_edge['edge_label'] = original_edge['edge_label']
    reverse_edge['edge_label_index'] = original_edge['edge_label_index'].index_select(0, torch.LongTensor([1, 0]))
    for key in original_edge.keys():
        if key not in ['edge_index', 'edge_attr', 'edge_label', 'edge_label_index']:
            reverse_edge[key] = original_edge[key]
                
    return reverse_edge

import pickle 

def get_datasets(get_edge_attr=False, filename=None, filter_top_k=False, top_k=50, remove_text_attr=True):
    if filename is None:
        filename = 'arxiv_author_paper_graph_training_v1.pkl'
    size = os.path.getsize(filename)
    print('size of dataset on disk: ', size/1e9, 'gb')

    if os.path.exists(filename):
        #data = HeteroData.from_dict(torch.load(filename))
        data = pickle.load(open(filename, 'rb'))
        print('loading saved heterodata object')


    
    def top_k_mask(scores, indices, top_k ):
        # Make sure we are using the GPU
        # scores = scores.cuda()
        # indices = indices.cuda()
        
        # Create an empty mask with the same shape as scores
        mask = torch.zeros_like(scores, dtype=torch.bool)
        # Get the unique indices and their counts
        unique_indices, counts = torch.unique(indices, return_counts=True)
    
        # Indices where count > top_k
        large_indices = unique_indices[counts > top_k]
    
        # Set mask for indices where count <= top_k
        mask[~torch.isin(indices,large_indices)] = True
        # For indices where count > 50, we only keep top 50 scores
        # if len(large_indices) > 0:
        #     idx_masks = [(indices == idx) for idx in tqdm(large_indices)]
        #     values_idxs = [scores[idx_mask].topk(top_k) for idx_mask in tqdm(idx_masks)]
        #     for idx_mask, (values, idxs) in tqdm(zip(idx_masks, values_idxs)):
        #         mask[idx_mask][idxs] = True
        
        
        # argsort the scores
        # first sort the numbers by the scores desceding
        # then sort the numbers in descending order such that the same numbers are neighbors but the  corresponding scores are still sorted in descending order
        # then get first top_k indices per number
        sorted_scores, sorted_score_indices = torch.sort(scores, descending=True)
        sorted_numbers = indices[sorted_score_indices]
        
        sorted_numbers_by_score_then_by_number, corresponding_mask = torch.sort(sorted_numbers, descending=True, stable=True)
        
        sorted_score_indices_by_score_then_by_number = sorted_score_indices[corresponding_mask]
        
        _, counts = torch.unique(sorted_numbers_by_score_then_by_number, sorted=True, return_counts=True)
        counts = torch.flip(counts, [0]) # descending
        start = torch.cat((torch.tensor([0]),torch.cumsum(counts, dim=0)[:-1]))
        end = start + top_k
        for s, e in zip(start, end):
            mask[sorted_score_indices_by_score_then_by_number[s:e]] = True
        
        # for idx in tqdm(large_indices):
            # mask[sorted_score_indices[(sorted_indices == idx)][:top_k]] = True
            
        # for idx in tqdm(large_indices):
        #     idx_mask = (sorted_indices == idx)
        #     reverse_map = sorted_score_indices[idx_mask][:top_k]
        #     mask[reverse_map] = True
            
        print('mask trues', torch.sum(mask), 'of total', mask.shape[0])
        return mask.cpu()

    
   
    if filter_top_k:
        def filter_once(edge, rev_e=None, keep_n_foreach_left=True, top_k=top_k):
            e = edge
            #rev_e = (e[2],'rev_'+e[1],e[0])
            
            if data[e].edge_attr.dim() == 1:
                data[e].edge_attr = data[e].edge_attr.unsqueeze(1)
            if rev_e is not None:
                if data[rev_e].edge_attr.dim() == 1:
                    data[rev_e].edge_attr = data[rev_e].edge_attr.unsqueeze(1)
            
            cache_dir = 'cache'
            if not os.path.exists(cache_dir):
                os.makedirs(cache_dir)
            
            mask_path = os.path.join(cache_dir, f'mask{top_k}_{"".join(list(edge))}.pt') 
            
            # if os.path.isfile(mask_path):
                # mask = torch.load(mask_path)
            # else:
            if keep_n_foreach_left:
                mask = top_k_mask(data[e].edge_attr.squeeze(1), data[e].edge_index[0,:], top_k)
            else: # for each right
                mask = top_k_mask(data[e].edge_attr.squeeze(1), data[e].edge_index[1,:], top_k)
                    
                # torch.save(mask, mask_path) 
            print('mask', mask.shape, data[e].edge_attr.shape, data[e].edge_index.shape)
            
            data[e].edge_attr = data[e].edge_attr[mask]
            data[e].edge_index = data[e].edge_index[:,mask]
            if rev_e is not None:
                data[rev_e].edge_attr = data[rev_e].edge_attr[mask]
                data[rev_e].edge_index = data[rev_e].edge_index[:,mask]
                
            print('keep',torch.sum(mask), 'of total',mask.shape[0])

        filter_once(('paper', 'has_word', 'word'), ('word', 'rev_has_word', 'paper'), keep_n_foreach_left=True) # for each paper only the top k words kept 
        filter_once(('word', 'co_occurs_with', 'word'), keep_n_foreach_left=True, top_k=top_k*2)
        filter_once(('word', 'co_occurs_with', 'word'), keep_n_foreach_left=False, top_k=top_k*2)
    
    data[('word','rev_co_occurs_with','word')].edge_index = data['word','co_occurs_with','word'].edge_index.flip(0)
    data[('word','rev_co_occurs_with','word')].edge_attr = data['word','co_occurs_with','word'].edge_attr
    
    
    from torch_geometric import seed_everything
    import torch_geometric.transforms as T
    from torch_geometric.utils import sort_edge_index


    
    edge_types = []
    rev_edge_types = []
    for edge_type in data.edge_types:
        print('len', edge_type, data[edge_type].edge_index.shape[1])
        if edge_type[1].startswith('rev_'):
            rev_edge_types.append(edge_type)
        else:
            edge_types.append(edge_type)

    
    transform = T.RandomLinkSplit(
        is_undirected=True,
        edge_types=edge_types,
        rev_edge_types=rev_edge_types,
        num_val=0.00,
        num_test=0.00,
        add_negative_train_samples=False, # only adds neg samples for val and test, neg train are added by LinkNeighborLoader. This means for each train batch, negs. are different, for val and train they stay the same
        neg_sampling_ratio=1.0,
        disjoint_train_ratio=0.3, #  training edges are shared for message passing and supervision
        )

    seed_everything(14)
    # sort by col to speed up sampling later (we can sepcify is_sorted=True in link neighbor loader)
    # we actually dont use the sort, because it seems to mess up things, but have not checked if everything works without sorting, so we leave it here
    def sort_edges(data):
        for edge_type in data.edge_types:
            if 'edge_attr' in data[edge_type].keys():
                data[edge_type].edge_index, data[edge_type].edge_attr = sort_edge_index(data[edge_type].edge_index, data[edge_type].edge_attr, sort_by_row=False) 
            else:
                data[edge_type].edge_index = sort_edge_index(data[edge_type].edge_index, sort_by_row=False) 
        return data
    

    def preprocess(data):
        if not get_edge_attr:
            # delete edge_attr of every edge type
            for edge_type in data.edge_types:
                del data[edge_type].edge_attr 

        # delete all keys for every node type except 'x' (e.g. description and title)
        for node_type in data.node_types:
            keys = list(data[node_type].keys())
            for key in keys:
                if key != 'x':
                    del data[node_type][key]
        return data
    
    
    # change all types to float32 and normalize the triangle columns
    for node_type in data.node_types:
        for i in range(data[node_type].x.shape[1]):
            if data[node_type].x[:,i].max()>5:
                #normalize
                print('normalizing column ', i, ' of node type ', node_type)
                data[node_type].x[:,i] = data[node_type].x[:,i]/data[node_type].x[:,i].max()
        
        data[node_type].x = data[node_type].x.to(torch.float32)
        
    
    
    
    train_data, val_data, test_data = transform(data)
    #train_data = sort_edges(train_data)
    #val_data = sort_edges(val_data)
    #test_data = sort_edges(test_data)
    if remove_text_attr:
        train_data = preprocess(train_data)
        val_data = preprocess(val_data)
        test_data = preprocess(test_data)
        preprocess(data)
    
    return train_data, val_data, test_data



# COMMAND ----------


from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.loader import HGTLoader
from torch_geometric.sampler import NegativeSampling

def get_hgt_linkloader(data, target_edge, batch_size, is_training, sampling_mode, neg_ratio, num_neighbors_hgtloader, num_workers, prefetch_factor, pin_memory):
    # first sample some edges in linkNeighborLoader
    # use the nodes of the sampled edges to sample from hgt loader
    
    
    num_neighbors_linkloader = [0]
    #for edge_type in data.edge_types:
    #    num_neighbors_linkloader[edge_type] = [0,0]
    
    negative_sampling = NegativeSampling(
        mode=sampling_mode, # binary or triplet
        amount=neg_ratio  # ratio, like Graphsage # 10
        #weight=  # "Probabilities" of nodes to be sampled: Node degree follows power law distribution
        )
    
    if sampling_mode == 'triplet':
        data[target_edge].edge_label = None
        

    linkNeighborLoader = LinkNeighborLoader(
            data,
            num_neighbors=num_neighbors_linkloader,
            edge_label_index=(target_edge, data[target_edge].edge_label_index), # if (edge, None), None means all edges are considered
        
            neg_sampling=negative_sampling, # adds negative samples
            batch_size=batch_size,
            shuffle=is_training, #is_training
            subgraph_type='directional', # contains only sampled edges
            #drop_last=True,
            num_workers=num_workers,
            #disjoint=True # sampled seed node creates its own, disjoint from the rest, subgraph, will add "batch vector" to loader output
        
            #num_workers=2,
            #prefetch_factor=2
            is_sorted = False,
            pin_memory=pin_memory,
            prefetch_factor=prefetch_factor,
    )
   
   
    def get_hgt(data, input_nodetype, input_mask):
        return next(iter(HGTLoader(
                data,
                # Sample 512 nodes per type and per iteration for 4 iterations
                num_samples=num_neighbors_hgtloader,
                batch_size=input_mask.shape[0],
                input_nodes=(input_nodetype, input_mask),
                num_workers=num_workers,
                pin_memory=pin_memory,
                prefetch_factor=prefetch_factor,
            )))
        
    
    def add_self_loops(data):
        for node_type in data.node_types:
            data[node_type, 'self_loop', node_type].edge_index = torch.arange(data[node_type].num_nodes).repeat(2,1)
        return data 

            
    def get_hgt_with_selfloops(loader):
        
        
        for batch in loader:   
            if sampling_mode=='triplet':      
                # original edge_label_index from the whole data object
                unmapped_batchids = torch.cat((batch[target_edge[0]].src_index,batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index.flatten()))
                original_node_ids = batch[target_edge[0]].n_id[unmapped_batchids]
                original_edge_label_nodes = torch.LongTensor(original_node_ids.unique())

                #remapping or sorting is not needed, since nodes are sorted, also in the htg batch, the edges will be the same
                src = batch[target_edge[0]].src_index.unsqueeze(0)
                src_total = src
                for i in range(neg_ratio):
                    src_total = torch.cat((src_total,src), dim=1)
                dst = torch.cat((batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index.flatten()),dim=0).unsqueeze(0)
                
                local_edge_label_index = torch.cat((src_total, dst),dim=0)
                edge_label = torch.cat((torch.ones(batch[target_edge[2]].dst_pos_index.shape[0]), torch.zeros(batch[target_edge[2]].dst_neg_index.flatten().shape[0])))
                
            elif sampling_mode=='binary':
                unmapped_batchids = batch[target_edge].edge_label_index.flatten()
                original_node_ids = batch[target_edge[0]].n_id[unmapped_batchids]
                original_edge_label_nodes = torch.LongTensor(original_node_ids.unique())

            else:
                raise Exception('binary or triplet sampling mode')
                
                
            hgt_batch = get_hgt(data, target_edge[0], original_edge_label_nodes) # 0,1,3,4,5,6,7,8,9,
          
            if sampling_mode=='triplet':
                
                # return message passing edges, and supervision edges/labels, ignore labels/label_indices in the message passing edges
                yield add_self_loops(hgt_batch), local_edge_label_index, edge_label, batch[target_edge].input_id, original_node_ids
            else: # sampling_mode=='binary':
                # return message passing edges, and supervision edges/labels, ignore labels/label_indices in the message passing edges, as well as original edge indices
                yield add_self_loops(hgt_batch), batch[target_edge].edge_label_index, batch[target_edge].edge_label, batch[target_edge].input_id, original_node_ids
    
    def get_hgt_2types_with_selfloops(loader):
        for batch in loader:
            if sampling_mode=='triplet':   
                original_src_nodes = batch[target_edge[0]].n_id[batch[target_edge[0]].src_index]
                original_edge_label_nodes_class1 = torch.LongTensor(original_src_nodes.unique())
                
                original_dst_nodes = batch[target_edge[2]].n_id[torch.cat((batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index.flatten()))]
                original_edge_label_nodes_class2 = torch.LongTensor(original_dst_nodes.unique())
                
                
                src = batch[target_edge[0]].src_index.unsqueeze(0)
                src_total = src
                for i in range(neg_ratio):
                    src_total = torch.cat((src_total,src), dim=1)
                
                dst = torch.cat((batch[target_edge[2]].dst_pos_index, batch[target_edge[2]].dst_neg_index.flatten()),dim=0).unsqueeze(0)
                
                local_edge_label_index = torch.cat((src_total, dst),dim=0)
                edge_label = torch.cat((torch.ones(batch[target_edge[2]].dst_pos_index.shape[0]), torch.zeros(batch[target_edge[2]].dst_neg_index.flatten().shape[0])))

            elif sampling_mode=='binary':
                original_src_nodes = batch[target_edge[0]].n_id[batch[target_edge].edge_label_index[0,:]]
                original_edge_label_nodes_class1 = original_src_nodes.unique()
                original_dst_nodes = batch[target_edge[2]].n_id[batch[target_edge].edge_label_index[1,:]]
                original_edge_label_nodes_class2 = original_dst_nodes.unique()

            else:
                raise Exception('binary or triplet sampling mode')

            # batch the start and end supervision nodes separately
            hgt_batch1 = get_hgt(data, target_edge[0], original_edge_label_nodes_class1)
            hgt_batch2 = get_hgt(data, target_edge[2], original_edge_label_nodes_class2)
            
            
            # ** We dont need to remove any edges ** since the supervision edges wont be sampled by hgt
            if sampling_mode=='triplet':
                yield add_self_loops(hgt_batch1), add_self_loops(hgt_batch2), local_edge_label_index, edge_label, batch[target_edge].input_id, original_src_nodes, original_dst_nodes
            else: # sampling_mode=='binary':
                # we can access the corresponding nodes of edge_label_index[0,:] in hgt_batch1[target_edge[0]], those of [1,:] in hgt_batch2...
                yield add_self_loops(hgt_batch1), add_self_loops(hgt_batch2), batch[target_edge].edge_label_index, batch[target_edge].edge_label, batch[target_edge].input_id, original_src_nodes, original_dst_nodes

        
    if target_edge[0] == target_edge[2]:
        # same edge type, only need to sample once
        return get_hgt_with_selfloops(linkNeighborLoader)
    else:
        return get_hgt_2types_with_selfloops(linkNeighborLoader)


import random

def get_minibatch_count(data, batch_size):
    batches = []
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            continue
        batches.extend([edge_type for _ in range((data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size)])
        
    return len(batches)

def get_single_minibatch_count(data, batch_size, edge_type):
    return (data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size

from copy import deepcopy
def uniform_hgt_sampler(data, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory):

    # return batches from all edgetypes with each "edge" being drawn uniformly at random (but we translate the probabilities to batches), last batches of each edge type may be smaller than batch_size
    batches = []
    loaders = {}
    # only the non-reverse edge types for now
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            continue
        batches.extend([edge_type for _ in range((data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size)])
        loaders[edge_type]=get_hgt_linkloader(data, edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)
        
    # random.seed(14)
    random.shuffle(batches)
    # set a random random seed again (may affect creating the loaders later for a second epoch)
    random.seed()
    
    print('total batches:', len(batches))
    
    for target_edge_type in batches:
        if target_edge_type[0] == target_edge_type[2]:
            same_nodetype = True
        else:
            same_nodetype = False
        try:
            minibatch = next(loaders[target_edge_type])
        except StopIteration: # "reinit" iterator
            loaders[target_edge_type] = get_hgt_linkloader(data, target_edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)#iter(loaders[target_edge_type])
            minibatch = next(loaders[target_edge_type])
            pass

        yield same_nodetype, target_edge_type, minibatch
        
def sampler_for_init(data, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory):
    batchcount=[]
    batches=[]
    loaders = {}
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            continue
        batchcount.extend([edge_type for _ in range((data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size)])
        batches.append(edge_type)
        loaders[edge_type]=get_hgt_linkloader(data, edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)
        
    print('total batches:', len(batchcount))

    
    # set a random random seed again (may affect creating the loaders later for a second epoch)
    random.seed()
    
    for target_edge_type in batches:
        if target_edge_type[0] == target_edge_type[2]:
            same_nodetype = True
        else:
            same_nodetype = False
        try:
            minibatch = next(loaders[target_edge_type])
        except StopIteration: # "reinit" iterator
            loaders[target_edge_type] = get_hgt_linkloader(data, target_edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)#iter(loaders[target_edge_type])
            minibatch = next(loaders[target_edge_type])
            pass
            
        yield same_nodetype, target_edge_type, minibatch

def equal_edgeweight_hgt_sampler(data, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory):
    batchcount=[]
    batches=[]
    loaders = {}
    for edge_type in data.edge_types:
        if edge_type[1].startswith('rev_'):
            continue
        batchcount.extend([edge_type for _ in range((data[edge_type].edge_label_index.shape[1]+batch_size)//batch_size)])
        batches.append(edge_type)
        loaders[edge_type]=get_hgt_linkloader(data, edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)
        
    print('total batches:', len(batchcount))

    batches = random.choices(batches, k=len(batchcount)) 
    # set a random random seed again (may affect creating the loaders later for a second epoch)
    random.seed()
    
   
    
    for target_edge_type in batches:
        if target_edge_type[0] == target_edge_type[2]:
            same_nodetype = True
        else:
            same_nodetype = False
        try:
            minibatch = next(loaders[target_edge_type])
        except StopIteration: # "reinit" iterator
            loaders[target_edge_type] = get_hgt_linkloader(data, target_edge_type, batch_size, is_training, sampling_mode, neg_sampling_ratio, num_neighbors, num_workers, prefetch_factor, pin_memory)#iter(loaders[target_edge_type])
            minibatch = next(loaders[target_edge_type])
            pass
            
        yield same_nodetype, target_edge_type, minibatch

c:\Users\I549512\AppData\Local\miniconda3\envs\gnnpapersearch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data, _, _ = get_datasets(get_edge_attr=False, filter_top_k=True, top_k=50, remove_text_attr=True)

size of dataset on disk:  0.043352309 gb
loading saved heterodata object
mask trues tensor(43362) of total 57073
mask torch.Size([57073]) torch.Size([57073, 1]) torch.Size([2, 57073])
keep tensor(43362) of total 57073
mask trues tensor(426432) of total 871906
mask torch.Size([871906]) torch.Size([871906, 1]) torch.Size([2, 871906])
keep tensor(426432) of total 871906
mask trues tensor(319735) of total 426432
mask torch.Size([426432]) torch.Size([426432, 1]) torch.Size([2, 426432])
keep tensor(319735) of total 426432
len ('paper', 'written_by', 'author') 3067
len ('paper', 'has_category', 'category') 1502
len ('paper', 'has_word', 'word') 43362
len ('paper', 'has_titleword', 'word') 7354
len ('paper', 'in_journal', 'journal') 1000
len ('word', 'co_occurs_with', 'word') 319735
len ('author', 'rev_written_by', 'paper') 3067
len ('category', 'rev_has_category', 'paper') 1502
len ('word', 'rev_has_word', 'paper') 43362
len ('word', 'rev_has_titleword', 'paper') 7354
len ('journal', 'rev_in_

In [3]:
from torch_geometric.nn.kge import TransE

import math

import torch
import torch.nn.functional as F
from torch import Tensor

from torch_geometric.nn.kge import KGEModel

# adapted and taken from https://github.com/pyg-team/pytorch_geometric/blob/master/torch_geometric/nn/kge/transe.py

class TransE(KGEModel):
    r"""The TransE model from the `"Translating Embeddings for Modeling
    Multi-Relational Data" <https://proceedings.neurips.cc/paper/2013/file/
    1cecc7a77928ca8133fa24680a88d2f9-Paper.pdf>`_ paper.

    :class:`TransE` models relations as a translation from head to tail
    entities such that

    .. math::
        \mathbf{e}_h + \mathbf{e}_r \approx \mathbf{e}_t,

    resulting in the scoring function:

    .. math::
        d(h, r, t) = - {\| \mathbf{e}_h + \mathbf{e}_r - \mathbf{e}_t \|}_p

    .. note::

        For an example of using the :class:`TransE` model, see
        `examples/kge_fb15k_237.py
        <https://github.com/pyg-team/pytorch_geometric/blob/master/examples/
        kge_fb15k_237.py>`_.

    Args:
        num_nodes (int): The number of nodes/entities in the graph.
        num_relations (int): The number of relations in the graph.
        hidden_channels (int): The hidden embedding size.
        margin (int, optional): The margin of the ranking loss.
            (default: :obj:`1.0`)
        p_norm (int, optional): The order embedding and distance normalization.
            (default: :obj:`1.0`)
        sparse (bool, optional): If set to :obj:`True`, gradients w.r.t. to the
            embedding matrices will be sparse. (default: :obj:`False`)
    """
    def __init__(
        self,
        num_nodes: int,
        num_relations: int,
        hidden_channels: int,
        margin: float = 1.0,
        p_norm: float = 1.0,
        sparse: bool = False,
    ):
        super().__init__(num_nodes, num_relations, hidden_channels, sparse)

        self.p_norm = p_norm
        self.margin = margin

        self.reset_parameters()

    def reset_parameters(self):
        bound = 6. / math.sqrt(self.hidden_channels)
        torch.nn.init.uniform_(self.node_emb.weight, -bound, bound)
        torch.nn.init.uniform_(self.rel_emb.weight, -bound, bound)
        F.normalize(self.rel_emb.weight.data, p=self.p_norm, dim=-1,
                    out=self.rel_emb.weight.data)

    def forward(
        self,
        head_embeddings: Tensor,
        rel_type,
        tail_embeddings: Tensor,
    ) -> Tensor:
        #head = self.node_emb(head_index)
        rel = self.rel_emb(rel_type)  # Amos: only learn the relation embeddings, others are learned with GNN
        #tail = self.node_emb(tail_index)

        head = F.normalize(head_embeddings, p=self.p_norm, dim=-1)
        tail = F.normalize(tail_embeddings, p=self.p_norm, dim=-1)
        # Calculate *negative* TransE norm:
        negative_norm = -((head + rel) - tail).norm(p=self.p_norm, dim=-1)
        return negative_norm

    
    def get_embedding(self,
                      embedding,
                    #   rel_type,
                        # have_head_or_tail
                      ):
        # rel = self.rel_emb(rel_type)
        embedding = F.normalize(embedding, p=self.p_norm, dim=-1)
        # if have_head_or_tail == 'head':
        #     return embedding + rel
        # else:
        #     return embedding - rel
        return embedding
    
    def get_relationship_embedding(self,rel_type):
        return self.rel_emb(rel_type)
    
    
    def loss(
        self,
        head_embeddings: Tensor,
        rel_type: Tensor,
        tail_embeddings: Tensor,
        labels: Tensor, # labels 0 or 1
    ) -> Tensor:
        pos_mask = labels == 1
        neg_mask = labels == 0
        
        pos_score = self(head_embeddings[pos_mask], rel_type, tail_embeddings[pos_mask])
        neg_score = self(head_embeddings[neg_mask], rel_type, tail_embeddings[neg_mask])
        loss = F.margin_ranking_loss(
            pos_score,
            neg_score,
            target=torch.ones_like(pos_score), # 1 for similarity, -1 for dissimilarity
            margin=self.margin,
        )
        return loss

In [4]:
from torch_geometric.nn import HGTConv, Linear
import torch 

class HGT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads, num_layers, node_types, data_metadata):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data_metadata,
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict = {
            node_type: self.lin_dict[node_type](x).relu_()
            for node_type, x in x_dict.items()
        }

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return x_dict

In [5]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Model(torch.nn.Module):
    def __init__(self, gnn : torch.nn.Module, head :  torch.nn.Module, node_types, edge_types, ggn_output_dim, pnorm=1):
        super().__init__()
        # edge_type onehot lookup table with keys
        # node_type onehot lookup table with keys
        self.node_type_embedding = torch.nn.Embedding(len(node_types), ggn_output_dim) # hidden channels should be the output dim of gnn

        self.edge_types = edge_types
        for edge_type in edge_types:
            if edge_type[1].startswith('rev_'):
                self.edge_types.remove(edge_type)

        # create edge to int mapping
        self.edgeindex_lookup = {edge_type:torch.tensor(i)  for i, edge_type in enumerate(edge_types)}

        # hidden channels should be the output dim of gnn
        if head=='TransE':
            self.head = TransE(len(node_types), len(edge_types) , ggn_output_dim, p_norm= pnorm, margin=0.5)  # KGE head with loss function
        else:
            raise NotImplementedError

        self.gnn = gnn



    def forward(self, hetero_data1, target_edge_type, edge_label_index, edge_label, hetero_data2=None, get_head_fn='loss'):

        if hetero_data2 is not None:
            assert target_edge_type[0] != target_edge_type[2], 'when passing two data objects, the edge type has to contain two different node types'
            head_embeddings = self.gnn(hetero_data1.x_dict, hetero_data1.edge_index_dict)[target_edge_type[0]][edge_label_index[0,:]]
            tail_embeddings = self.gnn(hetero_data2.x_dict, hetero_data2.edge_index_dict)[target_edge_type[2]][edge_label_index[1,:]]
        else:
            assert target_edge_type[0] == target_edge_type[2], 'when passing one data object, the edge type has to contain the same node types'


            embeddings = self.gnn(hetero_data1.x_dict, hetero_data1.edge_index_dict)
            head_embeddings = embeddings[target_edge_type[0]][edge_label_index[0,:]]
            tail_embeddings = embeddings[target_edge_type[2]][edge_label_index[1,:]]
        

        edgeindex = self.edgeindex_lookup[target_edge_type]
        if get_head_fn=='loss':
            loss = self.head.loss(head_embeddings, edgeindex.to(device), tail_embeddings, edge_label)
            return loss
        elif get_head_fn=='forward':
            return self.head.forward(head_embeddings, edgeindex.to(device), tail_embeddings)
        
    def get_embeddings(self, hetero_data, node_type, first_n_nodes):
        embeddings = self.gnn(hetero_data.x_dict, hetero_data.edge_index_dict)[node_type][:first_n_nodes,:]
        embeddings = self.head.get_embedding(embeddings)
        return embeddings

    def get_relationship_embedding(self, edge_type):
        return self.head.get_relationship_embedding(model.edgeindex_lookup[edge_type].cuda())

metadata = data.metadata()
# add selfloops
for node_type in data.node_types:
    metadata[1].append((node_type, 'self_loop', node_type))

out_channels = 256
hidden_channels = 256
num_heads = 8
num_layers = 2
pnorm = 2
head = 'TransE'
gnn = HGT(hidden_channels=out_channels, out_channels=out_channels, num_heads=num_heads, num_layers=num_layers, node_types=data.node_types, data_metadata=metadata)

model = Model(gnn, head=head, node_types=metadata[0], edge_types=metadata[1], ggn_output_dim=out_channels, pnorm=pnorm)
model.to(device)
model.eval()

Model(
  (node_type_embedding): Embedding(5, 256)
  (head): TransE(5, num_relations=14, hidden_channels=256)
  (gnn): HGT(
    (lin_dict): ModuleDict(
      (paper): Linear(-1, 256, bias=True)
      (author): Linear(-1, 256, bias=True)
      (category): Linear(-1, 256, bias=True)
      (word): Linear(-1, 256, bias=True)
      (journal): Linear(-1, 256, bias=True)
    )
    (convs): ModuleList(
      (0-1): 2 x HGTConv(-1, 256, heads=8)
    )
    (lin): Linear(256, 256, bias=True)
  )
)

In [6]:
# COMMAND ----------
# init model
from tqdm.auto import tqdm
from datetime import datetime
batch_size = 32
num_node_types = len(data.node_types)
print('num_node_types', num_node_types)
one_hop_neighbors = (20 * batch_size)//num_node_types # per relationship type
two_hop_neighbors = (20 * 8 * batch_size)//num_node_types # per relationship type
three_hop_neighbors = (20 * 8 * 3 * batch_size)//num_node_types # per relationship type
num_neighbors = [one_hop_neighbors, two_hop_neighbors] # three_hop_neighbors

from torch_geometric.loader import HGTLoader

num_node_types 5


In [7]:

# init
model.eval()
for node_type in data.node_types:
    print(node_type)
    
    loader = HGTLoader(
            data,
            # Sample 512 nodes per type and per iteration for 4 iterations
            num_samples=num_neighbors,
            batch_size=64, #96 or 32 nodes
            input_nodes=node_type,
            num_workers=0,
            pin_memory=True,
            prefetch_factor=None,
        )
    minibatch = next(iter(loader))
    
    model.get_embeddings(minibatch.cuda(), node_type, first_n_nodes=64)

paper
author
category
word
journal


In [8]:
minibatch

HeteroData(
  paper={
    x=[95, 393],
    n_id=[95],
  },
  author={
    x=[198, 385],
    n_id=[198],
  },
  category={
    x=[52, 385],
    n_id=[52],
  },
  word={
    x=[1024, 386],
    n_id=[1024],
  },
  journal={
    x=[64, 385],
    n_id=[64],
    input_id=[64],
    batch_size=64,
  },
  (paper, written_by, author)={
    edge_index=[2, 205],
    edge_label=[920],
    edge_label_index=[2, 920],
    e_id=[205],
  },
  (paper, has_category, category)={
    edge_index=[2, 93],
    edge_label=[450],
    edge_label_index=[2, 450],
    e_id=[93],
  },
  (paper, has_word, word)={
    edge_index=[2, 1819],
    edge_label=[13008],
    edge_label_index=[2, 13008],
    e_id=[1819],
  },
  (paper, has_titleword, word)={
    edge_index=[2, 463],
    edge_label=[2206],
    edge_label_index=[2, 2206],
    e_id=[463],
  },
  (paper, in_journal, journal)={
    edge_index=[2, 58],
    edge_label=[300],
    edge_label_index=[2, 300],
    e_id=[58],
  },
  (word, co_occurs_with, word)={
    edge_i

In [9]:
# load state dict
import os 
paths = os.listdir('cache/runs')[0]
path = os.path.join('cache/runs', paths)

for file in os.listdir(path):
    if file.endswith('.pt'):
        modelpath = os.path.join(path, file)
        break
print(modelpath)
model_and_optimizer = torch.load(modelpath)
model.load_state_dict(model_and_optimizer['model_state_dict'])

cache/runs\all_hgt_20240113_003259_mg05_norm2_lr0_0002_bs32_neigh_128_1024_h_TransE_hid_256_out_256_numh_8_numl_2\model_samplesseen0.pt


<All keys matched successfully>

In [10]:
minibatch['author']

{'x': tensor([[-0.0463, -0.0190,  0.0019,  ..., -0.0213, -0.0091,  0.1429],
        [-0.0730,  0.0219,  0.0578,  ...,  0.0144, -0.0317,  0.1429],
        [-0.0953, -0.0153, -0.0765,  ..., -0.0038, -0.0334,  0.1429],
        ...,
        [-0.0971, -0.0242,  0.0127,  ...,  0.0251,  0.0023,  0.1429],
        [-0.0787,  0.0635, -0.0321,  ..., -0.0167,  0.0337,  0.1429],
        [-0.0336, -0.0220, -0.0037,  ...,  0.0574, -0.0519,  0.1429]],
       device='cuda:0'), 'n_id': tensor([1614, 1220,  913, 2738, 1435,    0, 2494, 1663, 2739,  121,    1,   81,
           2, 2496, 1219, 1631, 1434,  502, 1736,   46,  501,   42,   22, 2501,
         971,  910,   67, 2034,  974,  970, 2498,  976,  911,  975, 1969,  909,
         973,  969, 1732, 1737, 1733, 1734, 1634,  116, 1640,   14,  204, 2622,
          12,   13,  914, 1533,  396, 1456, 1537, 1534,   28, 1423, 2499, 2884,
          45, 2296, 1917, 2502, 2504, 2645, 2500, 2495,  149, 2221, 1641, 1639,
        1632, 1629, 1635, 1633, 1637, 1636, 168

In [ ]:
model.eval()
with torch.no_grad():
    for node_type in data.node_types:
        batch_size = {
            'paper': 32,
            'author': 64,
            'word': 64,
            'journal': 64,
            'category': 64,
            }[node_type]
        
        print(node_type)
        
        
        loader = HGTLoader(
                data,
                # Sample 512 nodes per type and per iteration for 4 iterations
                num_samples=num_neighbors,
                batch_size=batch_size, #96 or 32 nodes
                input_nodes=node_type,
                num_workers=2,
                pin_memory=True,
                prefetch_factor=2,
                drop_last=False,
                shuffle=True
            )
        
        embeddings = []
        indices = []
        for minibatch in tqdm(loader):
            embds = model.get_embeddings(minibatch.cuda(), node_type, first_n_nodes=batch_size).cpu()
            n_ids = minibatch[node_type].n_id[:batch_size]
            embeddings.append(embds)
            indices.append(n_ids)
            # for i, n_id in enumerate(n_ids):
                # embeddings[n_id.item()] = embds[i,:]
        embeddings = torch.cat(embeddings)
        indices = torch.cat(indices)
        torch.save({'embeddings':embeddings, 'indices':indices}, f'embeddings_{node_type}.pt')
        
    for edge_type in data.edge_types:
        if 'rev_' in edge_type[1]:
            continue
        
        rel_embd = model.get_relationship_embedding(edge_type)
        torch.save({'embedding':edge_type}, f'relationship_embedding_{"_".join(edge_type)}.pt')